**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:

import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json,Model
from keras.layers.core import Dense,Lambda
from keras.optimizers import sgd
from keras.optimizers import adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Dropout,Flatten,Input

In [5]:
pip install scikit-video


In [0]:
#pip install openCV

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act implements an epsilon-greedy policy. With a probability equal to epsilon, we choose an action at random. This is called exploration. With a probability equal to 1 - epsilon we choose the best known action. This is called exploitation. Epsilon is essential because it ensures that we keep exploring the environment and that we don't keep choosing the same best known action at a time which can be suboptimal.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train= 10 # set small when debugging
epochs_test= 10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The arrays position and board are two attributes of the class environment. The array position stores the position of the agent in the environment during the game as well as the limits of the environment. The array board stores the rewards that the agent gets when is reaches the corresponding location in the environment.



## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(0,4)
        return action


***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        game_over = False
        win = 0
        lose = 0        

        while game_over == False:
            
            #play and interact
            action = agent.learned_act(state)
            state, reward, game_over = env.act(action)
            
            #update score
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
                
            # Save as a mp4
            env.draw(prefix+str(e))
            
        score = score + win-lose
        print("Win/lose count {}/{}. Average score ({})".format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [13]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.5/14.0. Average score (-2.5)
Win/lose count 12.5/17.0. Average score (-3.5)
Win/lose count 8.5/17.0. Average score (-5.166666666666667)
Win/lose count 6.0/9.0. Average score (-4.625)
Win/lose count 6.0/7.0. Average score (-3.9)
Win/lose count 7.0/8.0. Average score (-3.4166666666666665)
Win/lose count 9.0/14.0. Average score (-3.642857142857143)
Win/lose count 13.0/20.0. Average score (-4.0625)
Win/lose count 8.0/10.0. Average score (-3.8333333333333335)
Win/lose count 11.0/21.0. Average score (-4.45)
Final score: -4.45


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}






 1) Lets consider a policy $\pi$. We start with the definition of the Q function :
 
\begin{equation*}
Q^{\pi}(s,a) = E_{p}[\sum\limits_{t = 0}^{\infty}\gamma^{t}r(s_{t},a_{t})\|s_0=s,a_0=a]
\end{equation*}
We perform a one step look-ahead by removing the first term of the sum and rewriting the expectation. Since we consider Markov decision process :

\begin{equation*}
Q^{\pi}(s,a) =E_{p^{\pi}}[\gamma^0r(s,a) + \gamma E_{p^{\pi}}[\sum\limits_{t = 0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_0=s',a_0=a']
\end{equation*}
We recognize the following expression :
\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}


Lets consider a policy $\pi$. We start with the definition of the Q function :
\begin{equation*}
Q^{\pi}(s,a) = E_{p}[\sum\limits_{t = 0}^{\infty}\gamma^{t}r(s_{t},a_{t})\|s_0=s,a_0=a]
\end{equation*}
We perform a one step look-ahead by removing the first term of the sum and rewriting the expectation. Since we consider Markov decision process :

\begin{equation*}
Q^{\pi}(s,a) =E_{p^{\pi}}[\gamma^0r(s,a) + \gamma E_{p^{\pi}}[\sum\limits_{t = 0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_0=s',a_0=a']
\end{equation*}
We recognize the following expression :
\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}







2) For any policy $\pi$ we have demonstrated the following expression :

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

This expectation can be rewritten in the follwing form :



\begin{equation*}Q^{\pi}(s,a)= r(s,a) + \gamma\sum\limits_{s'\in S}p(s'|s,a)Q(s',\pi(s')) \end{equation*}



where $p(s'|s,a)$ represents the probability of transitionning from state $s$ to state $s'$ when selecting action $a$.

We know by definition that an optimal policy verifies the equation :



\begin{equation*}\pi^*(s)=\underset{a}{argmax}(Q^*(s,a))\end{equation*}



We deduce from the previous two equation that :



\begin{equation*}Q(s,\pi^*(s))=\underset{a \in A}{max}(Q(s,a))\end{equation*}





\begin{equation*}Q^{*}(s,a)= r(s,a) + \gamma\sum\limits_{s'\in S}p(s'|s,a)\underset{a'}{max}Q^{*}(s',a')\end{equation*}


which is equivalent to :

\begin{equation*}Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\underset{a'}{max}Q^{*}(s',a')]\end{equation*}


3) The objective of our network is to learn an optimal function $Q(\theta)$ where $\theta$ is the network's set of parameters. We know from the previous questions that such a function verifies the equation :
\begin{equation*}
Q(s,a,\theta)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\underset{a'}{max}Q(s',a',\theta)]
\end{equation*}
Thus, we want to minimize the error between the value of $Q(\theta)$ given by our network and the target value given by the bellman optimiality equation. To this purpose, we can choose to minimize the average squared error. We take the squared error because its differentiable.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>=self.max_memory:
            self.memory[0]= m # if the memory is full, we replace the first element by m
        else:
            self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s]))[0])  
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        #  memorize the states, learn using the memory

        self.memory.remember([s_, n_s_, a_, r_, game_over_]) # Update memory
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state)) # Initialization
        target_q = np.zeros((self.batch_size, 4)) # Initialize targets
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_

            target_q[i] = self.model.predict(np.array([s_]))[0]  # q-function for the current state
            #  targets
            if game_over_: # If episode is over
                target_q[i, a_] = r_
            else: #  update target using bellman optimality equation
                target_q[i, a_] = r_ + self.discount * self.model.predict(n_s_.reshape(1, 5, 5, self.n_state)).max()

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(64,input_shape =(5,5,self.n_state),activation="relu"))
        model.add(Flatten())
        model.add(Dense(4, activation="linear"))

        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [17]:
epochs_train = 20

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))













Epoch 000/020 | Loss 0.0032 | Win/lose count 9.5/7.0 (2.5)
Epoch 001/020 | Loss 0.0027 | Win/lose count 7.0/5.0 (2.0)
Epoch 002/020 | Loss 0.0039 | Win/lose count 3.5/3.0 (0.5)
Epoch 003/020 | Loss 0.0081 | Win/lose count 13.0/3.0 (10.0)
Epoch 004/020 | Loss 0.0055 | Win/lose count 5.0/4.0 (1.0)
Epoch 005/020 | Loss 0.0032 | Win/lose count 11.5/2.0 (9.5)
Epoch 006/020 | Loss 0.0025 | Win/lose count 9.0/3.0 (6.0)
Epoch 007/020 | Loss 0.0118 | Win/lose count 5.0/5.0 (0.0)
Epoch 008/020 | Loss 0.0026 | Win/lose count 4.0/2.0 (2.0)
Epoch 009/020 | Loss 0.0021 | Win/lose count 3.5/3.0 (0.5)
Epoch 010/020 | Loss 0.0013 | Win/lose count 14.5/3.0 (11.5)
Epoch 011/020 | Loss 0.0673 | Win/lose count 9.0/3.0 (6.0)
Epoch 012/020 | Loss 0.0020 | Win/lose count 15.0/0 (15.0)
Epoch 013/020 | Loss 0.0015 | Win/lose count 5.0/1.0 (4.0)
Epoch 014/020 | Loss 0.0029 | Win/lose count 12.5/6.0 (6.5)
Epoch 015/020 | Loss 0.0026 | Win/lose count 5.0/2.0 (3.0)
Epoch 016/020 | Loss 0.0017 | Win/lose

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)

        model = Sequential()
        model.add(Conv2D(16, (3, 3), input_shape = (5, 5, self.n_state), activation='relu'))
        model.add(Conv2D(16, (3, 3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation="linear"))

        
        model.compile(adam(lr=lr, decay=1e-4, ), "mse")
        self.model = model

In [19]:
epochs_train = 20
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0122 | Win/lose count 2.0/1.0 (1.0)
Epoch 001/020 | Loss 0.0037 | Win/lose count 1.5/3.0 (-1.5)
Epoch 002/020 | Loss 0.0022 | Win/lose count 7.0/1.0 (6.0)
Epoch 003/020 | Loss 0.0066 | Win/lose count 5.0/6.0 (-1.0)
Epoch 004/020 | Loss 0.0041 | Win/lose count 5.5/4.0 (1.5)
Epoch 005/020 | Loss 0.0039 | Win/lose count 6.0/4.0 (2.0)
Epoch 006/020 | Loss 0.0041 | Win/lose count 5.0/4.0 (1.0)
Epoch 007/020 | Loss 0.0057 | Win/lose count 7.0/3.0 (4.0)
Epoch 008/020 | Loss 0.0042 | Win/lose count 10.5/8.0 (2.5)
Epoch 009/020 | Loss 0.0524 | Win/lose count 5.5/4.0 (1.5)
Epoch 010/020 | Loss 0.0034 | Win/lose count 10.5/3.0 (7.5)
Epoch 011/020 | Loss 0.0038 | Win/lose count 11.5/1.0 (10.5)
Epoch 012/020 | Loss 0.0084 | Win/lose count 5.5/5.0 (0.5)
Epoch 013/020 | Loss 0.0020 | Win/lose count 7.0/4.0 (3.0)
Epoch 014/020 | Loss 0.0013 | Win/lose count 9.5/1.0 (8.5)
Epoch 015/020 | Loss 0.0011 | Win/lose count 13.0/6.0 (7.0)
Epoch 016/020 | Loss 0.0448 | Win/lose count 11.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [31]:
epochs_test = 20
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 4.5/0. Average score (3.25)
Win/lose count 3.5/0. Average score (3.3333333333333335)
Win/lose count 4.0/0. Average score (3.5)
Win/lose count 0.5/0. Average score (2.9)
Win/lose count 1.5/0. Average score (2.6666666666666665)
Win/lose count 1.0/0. Average score (2.4285714285714284)
Win/lose count 2.5/0. Average score (2.4375)
Win/lose count 4.0/0. Average score (2.611111111111111)
Win/lose count 1.0/0. Average score (2.45)
Win/lose count 2.0/0. Average score (2.409090909090909)
Win/lose count 0.5/0. Average score (2.25)
Win/lose count 8.0/0. Average score (2.6923076923076925)
Win/lose count 0.5/0. Average score (2.5357142857142856)
Win/lose count 0.5/0. Average score (2.4)
Win/lose count 4.0/0. Average score (2.5)
Win/lose count 6.0/0. Average score (2.7058823529411766)
Win/lose count 6.0/0. Average score (2.888888888888889)
Win/lose count 0.5/0. Average score (2.763157894736842)
Win/lose count 3.0/0. Average scor

In [32]:
HTML(display_videos('cnn_test10.mp4'))

In [33]:
HTML(display_videos('fc_test10.mp4'))

The first observation we can make is that the CNN model seems to work better than the FC model.
However, both models suffer from the lack of exploration. It seems that the agent has the tendency to oscillate around its position. Inceasing epsilon to tackle this problem didn't improve the outcomes significantly for the two algorithms.
Changing the size of the batches didn't yield noticeable improvements.
Moreover, increasing the temperature resulted in the increase of the overall score. The lack of exploration was recurrent regardless of the experimentation made.




***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    coeff = 0.7
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            
            action = agent.act(state)
            if agent.epsilon*coeff > 0.1:
              agent.set_epsilon(agent.epsilon*coeff)          
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
                
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board<0,2] = 256
        b[self.x,self.y,:]= 256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""
        
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train: 
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        
        reward += self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
          
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [49]:
# Training
epochs_train = 15
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = .4, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/015 | Loss 0.0088 | Win/lose count 9.0/25.000000000000078 (-16.000000000000078)
Epoch 001/015 | Loss 0.0107 | Win/lose count 5.5/18.599999999999998 (-13.099999999999998)
Epoch 002/015 | Loss 0.0225 | Win/lose count 3.5/20.300000000000022 (-16.800000000000022)
Epoch 003/015 | Loss 0.0059 | Win/lose count 10.0/19.100000000000005 (-9.100000000000005)
Epoch 004/015 | Loss 0.0065 | Win/lose count 17.0/21.20000000000003 (-4.200000000000031)
Epoch 005/015 | Loss 0.0035 | Win/lose count 15.0/20.60000000000001 (-5.6000000000000085)
Epoch 006/015 | Loss 0.0047 | Win/lose count 13.5/14.699999999999967 (-1.1999999999999673)
Epoch 007/015 | Loss 0.0050 | Win/lose count 12.0/16.999999999999975 (-4.999999999999975)
Epoch 008/015 | Loss 0.0051 | Win/lose count 19.0/15.699999999999969 (3.300000000000031)
Epoch 009/015 | Loss 0.0030 | Win/lose count 19.0/14.399999999999968 (4.600000000000032)
Epoch 010/015 | Loss 0.0429 | Win/lose count 20.0/14.299999999999974 (5.700000000000026)
Epoch 011/015

In [50]:
# Evaluation
epochs_test = 15
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 3.5/19.0. Average score (-15.5)
Win/lose count 11.5/16.59999999999997. Average score (-10.299999999999985)
Win/lose count 16.0/15.099999999999962. Average score (-6.566666666666644)
Win/lose count 10.0/17.299999999999976. Average score (-6.749999999999977)
Win/lose count 12.5/14.599999999999964. Average score (-5.8199999999999745)
Win/lose count 3.5/19.200000000000003. Average score (-7.466666666666646)
Win/lose count 7.0/17.399999999999977. Average score (-7.885714285714265)
Win/lose count 9.5/16.99999999999997. Average score (-7.837499999999978)
Win/lose count 7.5/18.599999999999998. Average score (-8.199999999999982)
Win/lose count 7.5/18.199999999999992. Average score (-8.449999999999982)
Win/lose count 8.0/16.599999999999966. Average score (-8.463636363636343)
Win/lose count 9.0/17.39999999999998. Average score (-8.458333333333313)
Win/lose count 5.0/18.39999999999999. Average score (-8.83846153846152)
Win/lose count 10.5/15.499999999999961. Average score (-8.564285

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***